### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, create the Yugabyte Platform API client object.

In [ ]:
import os
import yb_platform_client
from yb_platform_client.api import provider_api, session_api
from yb_platform_client.model.provider import Provider
from yb_platform_client.model.region import Region
from pprint import pprint

platform_address = 'http://localhost:9000'
platform_api_key = os.getenv('YB_API_KEY')

api_client = yb_platform_client.ApiClient(yb_platform_client.Configuration(
    host = platform_address,
    api_key = {
        'apiKeyAuth': platform_api_key,
    }
))

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [ ]:
session_api = session_api.SessionApi(api_client)

try:
    session_info = session_api.get_session_info()
except yb_platform_client.ApiException as e:
    print("Error get_session_info: %s" % e)
    raise

customer_uuid = session_info.get('customer_uuid')
print('Customer UUID:\n%s' % customer_uuid)

### Define Provider

In this example, we will create a GCP provider. First, load in GCP service account credentials JSON file. Then, define a `Provider` object with the credentials and desired configuration.

In [ ]:
import json

gcp_creds_filename = os.getenv('GCE_CREDENTIALS_FILE_PATH')
gcp_creds_file = open(gcp_creds_filename, 'r')
gcp_creds = json.loads(gcp_creds_file.read())

new_provider = Provider(
    code="gcp",
    config={
        **gcp_creds,
        "YB_FIREWALL_TAGS": "cluster-server",
    },
    dest_vpc_id="yugabyte-network",
    name="my-gcp-provider",
    regions=[
        Region(
            code="us-central1",
            name="us-central1",
            zones=[],
        ),
    ],
    ssh_port=54422,
    air_gap_install=False,
)

### Create Provider

Make API call to provider endpoint to create new provider.

In [ ]:
provider_api = provider_api.ProviderApi(api_client)

try:
    api_response = provider_api.create_providers(customer_uuid, new_provider)
except yb_platform_client.ApiException as e:
    print('Error create_providers: %s' % e)
    raise

pprint(api_response)